### GROUP 3 Final Report

##### Qi Gui
##### Yichen Li
##### Junzhe Yin
##### Jiatian Ye

### PART1. Data cleaning

In [3]:
import os
import glob
import pandas as pd

In [4]:
pwd

'/home/hadoop/project-group-3'

In [5]:
# Start SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("group3_project").getOrCreate()

In [6]:
# Start Spark Context
from pyspark import SparkContext, SparkConf
sc = spark.sparkContext

In [7]:
spark

In [8]:
sc

<SparkContext master=yarn appName=group3_project>

In [20]:
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer,IndexToString, VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns

##### 1.First of all, we select all our data as csv format and convert them to parquest, which might reduce the loading time.

In [24]:
# Read all csv from 
df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdata-group3/alldata/*.csv')

In [28]:
import time
%%time
df.count()

CPU times: user 5.84 ms, sys: 887 µs, total: 6.73 ms
Wall time: 1min 18s


76714860

In [31]:
#save parquet format in s3 to improve effectiveness
df.write.parquet("s3a://bigdata-group3/mergedata/hardrive.parquet")

##### 2. Then, we import the data from s3 as parquet format.

In [10]:
#read hard drive data 
df1 = spark.read\
  .format('parquet')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdata-group3/mergedata/hardrive.parquet/*.parquet')

In [12]:
df1.count()

76714860

In [13]:
len(df1.columns)#the data has 105 number of columns

105

##### 3. Clean the data by founding and dropping the columns with too much null value.

In [21]:
# Count the NA for each columns
df1.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df1.columns)).show()

+----+-------------+-----+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---------------

After counting the missing value in each columns, we decide to drop off the columns with missing value over 50%, and the name of them are as below.

In [23]:
#After counting NA, we decide to 
drop_columns=[
    'smart_2_normalized',
    'smart_2_raw',
    'smart_8_normalized',
    'smart_8_raw',
    'smart_11_normalized',
    'smart_11_raw',
    'smart_13_normalized',
    'smart_13_raw',
    'smart_15_normalized',
    'smart_15_raw',
    'smart_22_normalized',
    'smart_22_raw',
    'smart_177_normalized',
    'smart_177_raw',
    'smart_179_normalized',
    'smart_179_raw',
    'smart_181_normalized',
    'smart_181_raw',
    'smart_182_normalized',
    'smart_182_raw',
    'smart_183_normalized',
    'smart_183_raw',
    'smart_184_normalized',
    'smart_184_raw',
    'smart_189_normalized',
    'smart_189_raw',
    'smart_191_normalized',
    'smart_191_raw',
    'smart_195_normalized',
    'smart_195_raw',
    'smart_196_normalized',
    'smart_196_raw',
    'smart_200_normalized',
    'smart_200_raw',
    'smart_201_normalized',
    'smart_201_raw',
    'smart_220_normalized',
    'smart_220_raw',
    'smart_222_normalized',
    'smart_222_raw',
    'smart_223_normalized',
    'smart_223_raw',
    'smart_224_normalized',
    'smart_224_raw',
    'smart_225_normalized',
    'smart_225_raw',
    'smart_226_normalized',
    'smart_226_raw',
    'smart_235_normalized',
    'smart_235_raw',
    'smart_250_normalized',
    'smart_250_raw',
    'smart_251_normalized',
    'smart_251_raw',
    'smart_252_normalized',
    'smart_252_raw',
    'smart_254_normalized',
    'smart_254_raw',   
    'smart_255_normalized',
    'smart_255_raw'
]
mydf_new=df1.drop(*drop_columns)

In [24]:
mydf_new.columns

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_normalized',
 'smart_197_raw',
 'smart_198_normalized',
 'smart_198_raw',
 'smart_199_normalized',
 'smart_199_raw',
 'smart_240_normalized',
 'smart_240_raw',
 'smart_241_normalized',
 'smart_241_raw',
 'smart_242_normalized',
 'smart_242_raw']

In [25]:
mydf_new.count()

76714860

In [26]:
sc.stop()